# **Experiment Objectives and Hypotheses**

## **Hypotheses:**



*   H1: Training separate regression models for each of the three identified clusters will improve performance (RMSE, MAE, R²) over a single unified model.

* H2: Clustering using Mahalanobis distance will yield more effective segments for modeling than Euclidean distance.

* H3: Training separate models for each season (Spring, Summer, Fall, Winter) will improve predictive accuracy by capturing seasonal variation in bike rental behavior.

* H4: Neural Network outperforms our baseline




## **Research Questions:**
* RQ1: Does cluster-based modeling improve regression performance compared to a single global model?

* RQ2: Which clustering distance metric (Mahalanobis vs. Euclidean) produces better downstream model performance?

* RQ3: Can season-specific models capture trends in bike demand better than a unified model?

* RQ4: Among XGBoost, neural networks, and cluster-wise regressors, which method yields the best performance?



## **Experimental Design:**
### Baseline Model:
XGBoost Regressor
### Target Variable:
Rented Bike Count
### Dataset:
Seoul Bike Sharing Demand
### Data Split:
80% training, 20% testing

### Method:
[Insert a brief description of your method here]

### Visualization:

[Insert if applicable]


### Performance Metrics:
RMSE, MAE, and R²

## **Expected Outcomes:**


*  [Insert expectations from experiments]
*  [Insert expectations from experiments]




# **Import Libraries**

include additional libraries for development in this section

In [ ]:
pip install ucimlrepo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
import math
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis
from ucimlrepo import fetch_ucirepo
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor


from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D


plt.style.use('seaborn-v0_8-whitegrid')
mpl.rcParams['axes.edgecolor'] = '#333333'
mpl.rcParams['axes.linewidth'] = 0.8
mpl.rcParams['xtick.color'] = '#333333'
mpl.rcParams['ytick.color'] = '#333333'


# **Import Data**

In [ ]:
# Fetch Seoul Bike Sharing Demand dataset from UCI ML Repository
seoul_bike_sharing_demand = fetch_ucirepo(id=560)

In [ ]:
# Data (as pandas dataframes)
X_original = seoul_bike_sharing_demand.data.features
y_original = seoul_bike_sharing_demand.data.targets


In [ ]:
# Print dataset information
print("Dataset Metadata:")
print(seoul_bike_sharing_demand.metadata)
print("\nVariable Information:")
print(seoul_bike_sharing_demand.variables)

In [ ]:
# Examine feature information
print("\nOriginal feature columns:")
print(X_original.columns.tolist())
print("\nOriginal target variable:")
print(y_original.columns.tolist())

# **Data Preprocessing**

**Data Preprocessing Steps**



*   Makes "Rented Bike Count" new target. Move former target from dataset api to features
*   Check for missing values
*  Transform date column to date time
*  Transform categorical features using one hot encoding
* Split dataset into test (80%) and training (20%)
* Create heatmap to identify correlated features
* Compute VIF (Variance Inflation Factor) to measure multicolinearity
* Define function to remove colinear features using a 0.8 Colinearity threshold
* Compare VIF before and after removal





In [ ]:
# Make 'Rented Bike Count' the new target if it exists

if 'Rented Bike Count' in X_original.columns:
    # Make 'Rented Bike Count' the new target
    y = X_original[['Rented Bike Count']]
    # Remove 'Rented Bike Count' from features
    X = X_original.drop('Rented Bike Count', axis=1)
    # Add original target to features
    X = pd.concat([X, y_original], axis=1)

else:
    # If 'Rented Bike Count' is already the target, just confirm
    print("'Rented Bike Count' is already the target variable.")
    y = y_original
    X = X_original

In [ ]:
print("\nNew feature columns:")
print(X.columns.tolist())
print("\nNew target variable:")
print(y.columns.tolist())

In [ ]:
# Check for missing values
print("\nMissing values in features:")
print(X.isnull().sum())
print("\nMissing values in target:")
print(y.isnull().sum())

In [ ]:
# 1. Convert date column to datetime and extract useful components. date format is DD/MM/YYYY
if 'Date' in X.columns:
    # Specify the correct date format as DD/MM/YYYY
    X['Date'] = pd.to_datetime(X['Date'], format='%d/%m/%Y')
    X['Year'] = X['Date'].dt.year
    X['Month'] = X['Date'].dt.month
    X['Day'] = X['Date'].dt.day
    X['DayOfWeek'] = X['Date'].dt.dayofweek
    X = X.drop('Date', axis=1)

In [ ]:
# 2. Convert categorical features to numeric using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

In [ ]:
X

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_df = pd.DataFrame(X_train, columns=X_train.columns)

correlation_matrix = X_train_df.corr()

plt.figure(figsize=(12, 8))
# Use the calculated correlation matrix in the heatmap
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, fmt='.2f')
plt.title("Feature Correlation Heatmap")
plt.show()

In [ ]:
# Function to calculate VIF (Variance Inflation Factor)
def calculate_vif(X):
    # Select only numeric columns to avoid issues with non-numeric data types
    X_numeric = X.select_dtypes(include=np.number)
    # Drop columns with any non-finite values if they exist (though the type error suggests something else)
    # X_numeric = X_numeric.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

    # Check if there are still columns left
    if X_numeric.shape[1] == 0:
        print("Warning: No numeric columns found for VIF calculation.")
        return pd.DataFrame(columns=["feature", "VIF"])

    vif_data = pd.DataFrame()
    vif_data["feature"] = X_numeric.columns
    # Ensure the values passed to variance_inflation_factor are finite and the correct type
    vif_data["VIF"] = [variance_inflation_factor(X_numeric.values, i) for i in range(X_numeric.shape[1])]
    return vif_data.sort_values("VIF", ascending=False)

# Function to remove collinear features based on correlation threshold
def remove_collinear_features(X, threshold=0.8):
    # Select only numeric columns before calculating correlation
    X_numeric = X.select_dtypes(include=np.number)

    corr_matrix = X_numeric.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Plot correlation matrix
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', annot=False,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.savefig('correlation_matrix.png')
    plt.show()

    # Find features with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    print(f"Dropping {len(to_drop)} collinear features: {', '.join(to_drop)}")

    # Return dataframe with collinear features removed (applied to original X)
    return X.drop(to_drop, axis=1)


In [ ]:
# Calculate VIF before feature removal
print("Variance Inflation Factors before colinearity removal:")
# Ensure only numeric columns are passed to calculate_vif
X_train_numeric = X_train.select_dtypes(include=np.number)
vif_before = calculate_vif(X_train_numeric)
print(vif_before.head(10))  # Show top 10 highest VIF values

# Remove collinear features
# The remove_collinear_features function now handles numeric selection internally for correlation
X_train_filtered = remove_collinear_features(X_train, threshold=0.8)
# Ensure test set has the same columns as the filtered training set
X_test_filtered = X_test[X_train_filtered.columns]

# Calculate VIF after feature removal
print("\nVariance Inflation Factors after collinearity removal:")
# Ensure only numeric columns are passed to calculate_vif
X_train_filtered_numeric = X_train_filtered.select_dtypes(include=np.number)
vif_after = calculate_vif(X_train_filtered_numeric)
print(vif_after.head(10))

In [ ]:
# Standardize the filtered features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filtered)
X_test_scaled = scaler.transform(X_test_filtered)

# **Baseline Model Development**

In [ ]:
# Baseline XGBoost model
print("Training baseline XGBoost regression model...")
baseline_model = xgb.XGBRegressor(random_state=42)
baseline_model.fit(X_train_scaled, y_train.values.ravel())
baseline_preds = baseline_model.predict(X_test_scaled)
baseline_metrics = {
    'rmse': math.sqrt(mean_squared_error(y_test, baseline_preds)),
    'r2': r2_score(y_test, baseline_preds)
}

# **Model Development**

# **Evaluation and Visualizations**

In [ ]:
# Create a bar chart comparing the metrics
plt.figure(figsize=(12, 6))
fig.patch.set_facecolor('#f8f9fa')

# Define the colors from our palette
colors = ['#39A0ED', '#FF5E5B', '#D8D8D8']  # Blue, Coral, Light Gray

# RMSE comparison (lower is better)
plt.subplot(1, 2, 1)
models = ['Baseline', 'Model_developed_1', 'Model_developed_2']
rmse_values = [baseline_metrics['rmse'], Model_developed_1['rmse'], Model_developed_2['rmse']]
bars = plt.bar(models, rmse_values, color=colors,
               edgecolor='white', linewidth=0.8, width=0.7)

# Add values on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
            f'{height:.2f}', ha='center', va='bottom', fontweight='bold')


plt.title('RMSE Comparison (lower is better)', fontsize=16, pad=20, fontweight='bold', color='#333333')
plt.ylabel('RMSE', fontsize=12, color='#333333')
plt.grid(axis='y', alpha=0.3)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)


min_rmse = min(rmse_values)
plt.axhline(y=min_rmse, color='#333333', linestyle='--', alpha=0.5)
plt.text(len(models)-1, min_rmse-0.5, f'Best: {min_rmse:.2f}',
         ha='right', va='top', color='#333333', alpha=0.7)

In [ ]:
# R² comparison (higher is better)
plt.subplot(1, 2, 2)
r2_values = [baseline_metrics['r2'], Model_developed_1['r2'], Model_developed_2['r2']]
bars = plt.bar(models, r2_values, color=colors,  # Using the same color palette
               edgecolor='white', linewidth=0.8, width=0.7)

# Add values on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
            f'{height:.4f}', ha='center', va='bottom', fontweight='bold')

# Customize appearance
plt.title('R² Comparison (higher is better)', fontsize=16, pad=20, fontweight='bold', color='#333333')
plt.ylabel('R²', fontsize=12, color='#333333')
plt.grid(axis='y', alpha=0.3)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add a horizontal line at the maximum R² for reference
max_r2 = max(r2_values)
plt.axhline(y=max_r2, color='#333333', linestyle='--', alpha=0.5)
plt.text(len(models)-1, max_r2-0.02, f'Best: {max_r2:.4f}',
         ha='right', va='top', color='#333333', alpha=0.7)

plt.tight_layout()
plt.savefig('seoul_bike_metrics_comparison.png', dpi=300, bbox_inches='tight', facecolor='#f8f9fa')
plt.show()

In [ ]:
# MAE comparison (lower is better)
plt.subplot(1, 2, 2)

# Calculate MAE for each model
from sklearn.metrics import mean_absolute_error
baseline_mae = mean_absolute_error(y_test, baseline_preds)
Model_developed_1_mae = mean_absolute_error(y_test, Model_developed_1_preds)
Model_developed_2_mae = mean_absolute_error(y_test, Model_developed_2_preds)

mae_values = [baseline_mae, Model_developed_1, Model_developed_2]
bars = plt.bar(models, mae_values, color=colors,  # Using the same color palette
               edgecolor='white', linewidth=0.8, width=0.7)

# Add values on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
            f'{height:.2f}', ha='center', va='bottom', fontweight='bold')


plt.title('MAE Comparison (lower is better)', fontsize=16, pad=20, fontweight='bold', color='#333333')
plt.ylabel('MAE', fontsize=12, color='#333333')
plt.grid(axis='y', alpha=0.3)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add a horizontal line at the minimum MAE for reference
min_mae = min(mae_values)
plt.axhline(y=min_mae, color='#333333', linestyle='--', alpha=0.5)
plt.text(len(models)-1, min_mae-0.5, f'Best: {min_mae:.2f}',
         ha='right', va='top', color='#333333', alpha=0.7)

plt.tight_layout()
plt.savefig('seoul_bike_metrics_comparison.png', dpi=300, bbox_inches='tight', facecolor='#f8f9fa')
plt.show()